In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from tensorflow.keras.utils import to_categorical

In [2]:
filepath = "../input_data/streeteasy.csv"
data = pd.read_csv(filepath)
data.head(3)

,rental_id,building_id,rent,bedrooms,bathrooms,size_sqft,min_to_subway,floor,building_age_yrs,no_fee,has_roofdeck,has_washer_dryer,has_doorman,has_elevator,has_dishwasher,has_patio,has_gym,neighborhood,submarket,borough
0,1545,44518357,2550,0.0,1,480,9,2.0,17,1,1,0,0,1,1,0,1,Upper East Side,All Upper East Side,Manhattan
1,2472,94441623,11500,2.0,2,2000,4,1.0,96,0,0,0,0,0,0,0,0,Greenwich Village,All Downtown,Manhattan
2,10234,87632265,3000,3.0,1,1000,4,1.0,106,0,0,0,0,0,0,0,0,Astoria,Northwest Queens,Queens


In [3]:
selected_features = ["bedrooms","bathrooms","size_sqft","min_to_subway","floor","building_age_yrs","no_fee",
                    "has_roofdeck","has_washer_dryer","has_doorman","has_dishwasher","has_patio","has_gym","neighborhood",
                     "submarket","borough"]



#selected_features = ["bedrooms","bathrooms","size_sqft","neighborhood"]
selected_features

['bedrooms',
 'bathrooms',
 'size_sqft',
 'min_to_subway',
 'floor',
 'building_age_yrs',
 'no_fee',
 'has_roofdeck',
 'has_washer_dryer',
 'has_doorman',
 'has_dishwasher',
 'has_patio',
 'has_gym',
 'neighborhood',
 'submarket',
 'borough']

In [4]:
y = data["rent"]
X = data[selected_features]

In [5]:
X=pd.get_dummies(X)
X.head(3)

,bedrooms,bathrooms,size_sqft,min_to_subway,floor,building_age_yrs,no_fee,has_roofdeck,has_washer_dryer,has_doorman,...,submarket_Northeast Queens,submarket_Northwest Brooklyn,submarket_Northwest Queens,submarket_Prospect Park,submarket_South Brooklyn,submarket_South Queens,submarket_The Rockaways,borough_Brooklyn,borough_Manhattan,borough_Queens
0,0.0,1,480,9,2.0,17,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2.0,2,2000,4,1.0,96,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3.0,1,1000,4,1.0,106,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=13)

In [13]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#y_train_scaled = y_scaler.transform(y_train)
#y_test_scaled = y_scaler.transform(X_test)

ValueError: Expected 2D array, got 1D array instead:
array=[4480. 4850. 3700. ... 1800. 5317. 2123.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [8]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model = knn_model.fit(X_train, y_train)

print(f"Training Data Score: {knn_model.score(X_train, y_train)}")
print(f"Testing Data Score: {knn_model.score(X_test, y_test)}")

Training Data Score: 0.2384
Testing Data Score: 0.032


In [9]:
tree_model = tree.DecisionTreeClassifier()
tree_model = tree_model.fit(X_train,y_train)
tree_model.score(X_train,y_train)

0.9933333333333333

In [10]:
predictions = tree_model.predict(X_test)
results = pd.DataFrame({"predicted":predictions,"actual":y_test, "accuracy":1-abs(predictions-y_test)/y_test})
results.head(5)

,predicted,actual,accuracy
1552,2150,2000,0.925000
658,1550,2400,0.645833
1763,3800,3600,0.944444
2272,7995,10000,0.799500
1404,2900,2500,0.840000


In [11]:
results.mean()

predicted    4559.658400
actual       4600.616800
accuracy        0.813538
dtype: float64

In [12]:
inputs = [1,1,900,"Midtown"]

neighborhoods = []
submarkets = []
boroughs = []

n_list_index = "na"
sm_list_index = "na"
b_list_index= "na"

if "neighborhood" in selected_features:
    n_list_index = selected_features.index("neighborhood")
    neighborhood_list = data.neighborhood.unique()
    for neighborhood in neighborhood_list:
        if neighborhood == inputs[n_list_index]:
            neighborhoods.append(1)
        else:
            neighborhoods.append(0)
if "submarket" in selected_features:
    sm_list_index = selected_features.index("submarket")
    submarket_list = data.submarket.unique()
    for submarket in submarket_list:
        if submarket == inputs[sm_list_index]:
            submarkets.append(1)
        else:
            submarkets.append(0)
if "borough" in selected_features:
    b_list_index = selected_features.index("borough")
    borough_list = data.borough.unique()
    for borough in borough_list:
        if borough == inputs[b_list_index]:
            submarkets.append(1)
        else:
            submarkets.append(0)

stop_index = "na"
if n_list_index != "na":
    stop_index = n_list_index
elif sm_list_index != "na":
    stop_index = sm_list_index
elif b_list_index != "na":
    stop_index = b_list_index

value_inputs = []
if stop_index == "na":
    for value in inputs:
        value_inputs.append([value])
elif stop_index != "na":
    for x in range(int(stop_index)-0):
        value_inputs.append(inputs[x])

test_input = value_inputs+neighborhoods+submarkets+boroughs
test_input

IndexError: list index out of range

In [ ]:
tree_model.predict([test_input])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(n_estimators=200)
random_forest_model = random_forest_model.fit(X_train, y_train)

print(f"Training Data Score: {random_forest_model.score(X_train, y_train)}")
print(f"Testing Data Score: {random_forest_model.score(X_test, y_test)}")